# How to Make a Self-Driving Car

Udacity recently open sourced their self driving car simulator
originally built for SDND students

![alt text](https://github.com/udacity/self-driving-car-sim/raw/master/sim_image.png "Logo Title Text 1")

- built in Unity (free game making engine https://unity3d.com/)
- add new tracks, change prebuilt scripts like gravity acceleration easily

Code:
https://github.com/udacity/self-driving-car-sim
    
## Data Generation 

- records images from center, left, and right cameras w/ associated steering angle, speed, throttle and brake. 
- saves to CSV


## Training Mode - Behavioral cloning

We use a 9 layer convolutional network, 



#### Hardware design:

![alt text](https://devblogs.nvidia.com/parallelforall/wp-content/uploads/2016/08/data-collection-system.png "Logo Title Text 1")

- 3 cameras
-  The steering command is obtained by tapping into the vehicle’s Controller Area Network (CAN) bus.
- Nvidia's Drive PX onboard computer with GPUs

In order to make the system independent of the car geometry, the steering command is 1/r, where r is the turning radius in meters.  1/r was used instead of r to prevent a singularity when driving straight (the turning radius for driving straight is infinity). 1/r smoothly transitions through zero from left turns (negative values) to right turns (positive values).


#### Software Design (supervised learning!) :

![alt text](https://devblogs.nvidia.com/parallelforall/wp-content/uploads/2016/08/training.png "Logo Title Text 1")

Images are fed into a CNN that then computes a proposed steering command. The proposed command is compared to the desired command for that image, and the weights of the CNN are adjusted to bring the CNN output closer to the desired output. The weight adjustment is accomplished using back propagation

Eventually, it generated steering commands using just a single camera

![alt text](https://devblogs.nvidia.com/parallelforall/wp-content/uploads/2016/08/inference.png "Logo Title Text 1")

## Testing mode

We will just run autonomous mode, then run our model and the car will start driving

![alt text](https://cdn-images-1.medium.com/max/1440/1*nlusa_fC5BnsgnWPFnov7Q.tiff "Logo Title Text 1")



In [1]:
# Step 1 - Install dependencies

#TensorFlow without GPU
!conda env create -f environments.yml 



CondaValueError: prefix already exists: C:\Users\asus\.cache\Nouveau dossier\envs\car-behavioral-cloning



In [1]:
#loader = importlib._bootstrap_external.SourceLoader()



In [2]:
!conda activate car-behavioral-cloning

In [3]:
#Use TensorFlow with GPU
!conda env create -f environment-gpu.yml



CondaValueError: prefix already exists: C:\Users\asus\.cache\Nouveau dossier\envs\car-behavioral-cloning



In [4]:
import pandas as pd # data analysis toolkit - create, read, update, delete datasets
import numpy as np #matrix math
from sklearn.model_selection import train_test_split #to split out training and testing data 
#keras is a high level wrapper on top of tensorflow (machine learning library)
#The Sequential container is a linear stack of layers
from keras.models import Sequential
#popular optimization strategy that uses gradient descent 
from keras.optimizers import Adam
#to save our model periodically as checkpoints for loading later
from keras.callbacks import ModelCheckpoint
#what types of layers do we want our model to have?
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
#helper class to define input shape and generate training images given image paths & steering angles
from utils import INPUT_SHAPE, batch_generator
#for command line arguments
import argparse
#for reading files
import os
 

Using TensorFlow backend.


In [5]:
import cv2, os
import numpy as np
import matplotlib.image as mpimg


IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 160, 320, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)


def load_image(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))


def crop(image):
    """
    Crop the image (removing the sky at the top and the car front at the bottom)
    """
    return image[60:-25, :, :] # remove the sky and the car front


def resize(image):
    """
    Resize the image to the input shape used by the network model
    """
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)


def rgb2yuv(image):
    """
    Convert the image from RGB to YUV (This is what the NVIDIA model does)
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)


def preprocess(image):
    """
    Combine all preprocess functions into one
    """
    image = crop(image)
    image = resize(image)
    image = rgb2yuv(image)
    return image


def choose_image(data_dir, center, left, right, steering_angle):
    """
    Randomly choose an image from the center, left or right, and adjust
    the steering angle.
    """
    choice = np.random.choice(3)
    if choice == 0:
        return load_image(data_dir, left), steering_angle + 0.2
    elif choice == 1:
        return load_image(data_dir, right), steering_angle - 0.2
    return load_image(data_dir, center), steering_angle


def random_flip(image, steering_angle):
    """
    Randomly flipt the image left <-> right, and adjust the steering angle.
    """
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle
    return image, steering_angle


def random_translate(image, steering_angle, range_x, range_y):
    """
    Randomly shift the image virtially and horizontally (translation).
    """
    trans_x = range_x * (np.random.rand() - 0.5)
    trans_y = range_y * (np.random.rand() - 0.5)
    steering_angle += trans_x * 0.002
    trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, trans_m, (width, height))
    return image, steering_angle


def random_shadow(image):
    """
    Generates and adds random shadow
    """
    # (x1, y1) and (x2, y2) forms a line
    # xm, ym gives all the locations of the image
    x1, y1 = IMAGE_WIDTH * np.random.rand(), 0
    x2, y2 = IMAGE_WIDTH * np.random.rand(), IMAGE_HEIGHT
    xm, ym = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]

    # mathematically speaking, we want to set 1 below the line and zero otherwise
    # Our coordinate is up side down.  So, the above the line: 
    # (ym-y1)/(xm-x1) > (y2-y1)/(x2-x1)
    # as x2 == x1 causes zero-division problem, we'll write it in the below form:
    # (ym-y1)*(x2-x1) - (y2-y1)*(xm-x1) > 0
    mask = np.zeros_like(image[:, :, 1])
    mask[(ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0] = 1

    # choose which side should have shadow and adjust saturation
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    # adjust Saturation in HLS(Hue, Light, Saturation)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)


def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    """
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


def augument(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    """
    Generate an augumented image and adjust steering angle.
    (The steering angle is associated with the center image)
    """
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    image = random_shadow(image)
    image = random_brightness(image)
    return image, steering_angle


def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            # argumentation
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augument(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center) 
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers



## Generate data

In [6]:
#for debugging, allows for reproducible (deterministic) results 
np.random.seed(0)
def s2b(s):
    """
    Converts a string to boolean value
    """
    s = s.lower()
    return s == 'true' or s == 'yes' or s == 'y' or s == '1'



def load_data(args):
    """
    Load training data and split it into training and validation set
    """
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(os.path.join(os.getcwd(), args.data_dir, 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

    #yay dataframes, we can select rows and columns by their names
    #we'll store the camera images as our input data
    X = data_df[['center', 'left', 'right']].values
    #and our steering commands as our output data
    y = data_df['steering'].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args.test_size, random_state=0)

    return X_train, X_valid, y_train, y_valid


In [7]:
parser = argparse.ArgumentParser(description='Behavioral Cloning Training Program')
parser.add_argument('-d', help='data directory',        dest='data_dir',          type=str,   default='data')
parser.add_argument('-t', help='test size fraction',    dest='test_size',         type=float, default=0.2)
parser.add_argument('-k', help='drop out probability',  dest='keep_prob',         type=float, default=0.5)
parser.add_argument('-n', help='number of epochs',      dest='nb_epoch',          type=int,   default=1)
parser.add_argument('-s', help='samples per epoch',     dest='samples_per_epoch', type=int,   default=20000)
parser.add_argument('-b', help='batch size',            dest='batch_size',        type=int,   default=40)
parser.add_argument('-o', help='save best models only', dest='save_best_only',    type=s2b,   default='true')
parser.add_argument('-l', help='learning rate',         dest='learning_rate',     type=float, default=1.0e-4)

args, unknown = parser.parse_known_args()
#args = parser.parse_args()


In [8]:
args

Namespace(batch_size=40, data_dir='data', keep_prob=0.5, learning_rate=0.0001, nb_epoch=1, samples_per_epoch=20000, save_best_only=True, test_size=0.2)

In [9]:
 X_train, X_valid, y_train, y_valid= load_data(args)


In [10]:
data= load_data(args)


# Building model

In [11]:
def build_model(args):
    """
    NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)

    # the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 
    """
    model = Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(160, 320, 3)))
    model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Conv2D(64, 3, 3, activation='elu'))
    model.add(Dropout(args.keep_prob))
    model.add(Flatten())
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    model.add(Dense(1))
    model.summary()

    return model


def train_model(model, args, X_train, X_valid, y_train, y_valid):
    """
    Train the model
    """
    #Saves the model after every epoch.
    #quantity to monitor, verbosity i.e logging mode (0 or 1), 
    #if save_best_only is true the latest best model according to the quantity monitored will not be overwritten.
    #mode: one of {auto, min, max}. If save_best_only=True, the decision to overwrite the current save file is
    # made based on either the maximization or the minimization of the monitored quantity. For val_acc, 
    #this should be max, for val_loss this should be min, etc. In auto mode, the direction is automatically
    # inferred from the name of the monitored quantity.
    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=args.save_best_only,
                                 mode='auto')

    #calculate the difference between expected steering angle and actual steering angle
    #square the difference
    #add up all those differences for as many data points as we have
    #divide by the number of them
    #that value is our mean squared error! this is what we want to minimize via
    #gradient descent
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=args.learning_rate))

    #Fits the model on data generated batch-by-batch by a Python generator.

    #The generator is run in parallel to the model, for efficiency. 
    #For instance, this allows you to do real-time data augmentation on images on CPU in 
    #parallel to training your model on GPU.
    #so we reshape our data into their appropriate batches and train our model simulatenously
    model.fit_generator(batch_generator(args.data_dir, X_train, y_train, args.batch_size, True),
                        args.samples_per_epoch,
                        args.nb_epoch,
                        max_q_size=1,
                        validation_data=batch_generator(args.data_dir, X_valid, y_valid, args.batch_size, False),
                        nb_val_samples=len(X_valid),
                        callbacks=[checkpoint],
                        verbose=1)


In [70]:
#build model
model = build_model(args)


model.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
  
model.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
  def load_data(args):
model.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
  """
model.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  Load training data and split it into training and validation set
model.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  """


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_16 (Lambda)           (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 78, 158, 24)       1824      
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 37, 77, 36)        21636     
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 17, 37, 48)        43248     
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 15, 35, 64)        27712     
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 13, 33, 64)        36928     
_________________________________________________________________
dropout_16 (Dropout)         (None, 13, 33, 64)      

In [71]:
#train model on data, it saves as model.h5 
train_model(model, args,  X_train, X_valid, y_train, y_valid)


model.py:77: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  model.summary()
model.py:77: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., 20000, 1, validation_data=<generator..., callbacks=[<keras.ca..., verbose=1, validation_steps=327, max_queue_size=1)`
  model.summary()


Epoch 1/1
20000/20000 [==============================] - 33107s 2s/step - loss: 0.0413 - val_loss: 0.0660


In [72]:
from keras.models import model_from_json
# serialize model to JSON
#model_json = model.to_json()
#with open("model.json", "w") as json_file:
#    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("model.h5")
#print("Saved model to disk")

Saved model to disk


# Testing Model

In [12]:
#model.save("model")

In [21]:
#parsing command line arguments
import argparse
#decoding camera images
import base64
#for frametimestamp saving
from datetime import datetime
#reading and writing files
import os
#high level file operations
import shutil
#matrix math
import numpy as np
#real-time server
import socketio
#concurrent networking 
import eventlet
#web server gateway interface
import eventlet.wsgi
#image manipulation
from PIL import Image
#web framework
from flask import Flask
#input output
from io import BytesIO

#load our saved model
from keras.models import load_model

#helper class
import utils

#initialize our server
sio = socketio.Server()
#our flask (web) app
app = Flask(__name__)
#init our model and image array as empty
prev_image_array = None
model= None
#set min/max speed for our autonomous car
MAX_SPEED = 25
MIN_SPEED = 10

#and a speed limit
speed_limit = MAX_SPEED

#registering event handler for the server
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = float(data["steering_angle"])
        # The current throttle of the car, how hard to push peddle
        throttle = float(data["throttle"])
        # The current speed of the car
        speed = float(data["speed"])
        # The current image from the center camera of the car
        image = Image.open(BytesIO(base64.b64decode(data["image"])))
        try:
            image = np.asarray(image)       # from PIL image to numpy array
            image = utils.preprocess(image) # apply the preprocessing
            image = np.array([image])       # the model expects 4D array

            # predict the steering angle for the image
            steering_angle = float(model.predict(image, batch_size=1))
            # lower the throttle as the speed increases
            # if the speed is above the current speed limit, we are on a downhill.
            # make sure we slow down first and then go back to the original max speed.
            global speed_limit
            if speed > speed_limit:
                speed_limit = MIN_SPEED  # slow down
            else:
                speed_limit = MAX_SPEED
            throttle = 1.0 - steering_angle**2 - (speed/speed_limit)**2

            print('{} {} {}'.format(steering_angle, throttle, speed))
            send_control(steering_angle, throttle)
        except Exception as e:
            print(e)

        # save frame
        if args.image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(args.image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)




In [14]:
parser = argparse.ArgumentParser(description='Behavioral Cloning Training Program')
parser.add_argument('-d', help='data directory',        dest='data_dir',          type=str,   default='data')
parser.add_argument('-t', help='test size fraction',    dest='test_size',         type=float, default=0.2)
parser.add_argument('-k', help='drop out probability',  dest='keep_prob',         type=float, default=0.5)
parser.add_argument('-n', help='number of epochs',      dest='nb_epoch',          type=int,   default=1)
parser.add_argument('-s', help='samples per epoch',     dest='samples_per_epoch', type=int,   default=20000)
parser.add_argument('-b', help='batch size',            dest='batch_size',        type=int,   default=40)
parser.add_argument('-o', help='save best models only', dest='save_best_only',    type=s2b,   default='true')
parser.add_argument('-l', help='learning rate',         dest='learning_rate',     type=float, default=1.0e-4)
parser.add_argument(
        'model',
        type=str,
        help='Path to model h5 file. Model should be on the same path.'
    )



_StoreAction(option_strings=[], dest='model', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='Path to model h5 file. Model should be on the same path.', metavar=None)

In [15]:

parser.add_argument(
        'image_folder',
        type=str,
        nargs='?',
        default='',
        help='Path to image folder. This is where the images from the run will be saved.'
    )

_StoreAction(option_strings=[], dest='image_folder', nargs='?', const=None, default='', type=<class 'str'>, choices=None, help='Path to image folder. This is where the images from the run will be saved.', metavar=None)

In [16]:
args, unknown = parser.parse_known_args()

In [17]:
args

Namespace(batch_size=40, data_dir='data', image_folder='', keep_prob=0.5, learning_rate=0.0001, model='C:\\Users\\asus\\AppData\\Roaming\\jupyter\\runtime\\kernel-fcbbe17f-a054-47e1-b434-6dc83d6a02ed.json', nb_epoch=1, samples_per_epoch=20000, save_best_only=True, test_size=0.2)

In [ ]:

    if args.image_folder != '':
        print("Creating image folder at {}".format(args.image_folder))
        if not os.path.exists(args.image_folder):
            os.makedirs(args.image_folder)
        else:
            shutil.rmtree(args.image_folder)
            os.makedirs(args.image_folder)
        print("RECORDING THIS RUN ...1")
    else:
        print("NOT RECORDING THIS RUN ...2")
    from keras.models import model_from_json

    # load json and create model
    json_file = open('model.json', 'r')
    model = json_file.read()
    json_file.close()
    model = model_from_json(model)
    # load weights into new model
    model.load_weights("model1.h5")
    print("Loaded model from disk")



    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('',  4567)), app)
    


NOT RECORDING THIS RUN ...2
Loaded model from disk


(1612) wsgi starting up on http://0.0.0.0:4567
(1612) accepted ('127.0.0.1', 54228)


connect  ecd18ef699294295b20ccf118f63e502
-0.1690676212310791 0.9714161394512644 0.0
-0.1677882820367813 0.9718470924111455 0.0
-0.1687803715467453 0.9715131861805426 0.0
-0.16782338917255402 0.9717744700466375 0.195
-0.1659660041332245 0.9675210659360505 1.7561
-0.1659660041332245 0.9675210659360505 1.7561
-0.1909107118844986 0.954287112487754 2.4065
-0.13082140684127808 0.9658074516560688 3.2671
-0.11759142577648163 0.9583673610798542 4.1687
-0.11759142577648163 0.9583673610798542 4.1687
-0.14465878903865814 0.941555694337869 4.8424
-0.09460388123989105 0.9342879961503486 5.9562
-0.09097588062286377 0.9077439858808944 7.2448
-0.09097588062286377 0.9077439858808944 7.2448
-0.09642182290554047 0.8859988156675727 8.0895
-0.08303859829902649 0.8484917127925329 9.507
-0.07303430140018463 0.837609130402987 9.9076
-0.11693580448627472 0.8102148592931477 10.4914
-0.1085711419582367 0.7852800767338844 11.262
-0.1085711419582367 0.7562429237098843 12.0408
-0.1085711419582367 0.7562429237098843

-0.02162073738873005 0.19747617193876765 22.3894
-0.05458604171872139 0.19562300078548223 22.3802
-0.05458604171872139 0.19562300078548223 22.3802
-0.024920525029301643 0.19756617383226394 22.386
-0.011133107356727123 0.18419251881658338 22.5788
-0.022609131410717964 0.18337172151285264 22.5848
-0.022609131410717964 0.18337172151285264 22.5848
-0.03259653225541115 0.1821697885809218 22.5938
-0.047278497368097305 0.19547192242261469 22.3927
-0.047278497368097305 0.19547192242261469 22.3927
-0.007925393991172314 0.19833212315408466 22.3831
0.007624280638992786 0.19900994764073798 22.3737
-0.007094535045325756 0.18465624747649068 22.5734
-0.007094535045325756 0.18465624747649068 22.5734
0.04046344384551048 0.1826069208481612 22.5798
0.07313905656337738 0.17837456800501916 22.587
0.01948252134025097 0.1981084773782268 22.3818
0.01948252134025097 0.1981084773782268 22.3818
0.00843397993594408 0.1852276663184399 22.5652
0.008869078941643238 0.2001273860947269 22.3578
0.008869078941643238 0.2

-0.1148393303155899 0.18045103163666687 22.4494
0.011331257410347462 0.20806349570950056 22.2459
-0.06135334447026253 0.19076541672231317 22.437
-0.20493556559085846 0.166328556355955 22.244
-0.20493556559085846 0.166328556355955 22.244
-0.09835347533226013 0.18413091643406665 22.4471
-0.05774450674653053 0.19217855931659977 22.4233
-0.09168374538421631 0.1885056837283221 22.4038
-0.021494464948773384 0.21155510412056588 22.1921
0.005436909385025501 0.21173898504033917 22.1956
-0.21834968030452728 0.14954325671091073 22.3995
-0.12753893435001373 0.19451479582486297 22.2095
-0.12753893435001373 0.19451479582486297 22.2095
-0.13814415037631989 0.18022767837280462 22.3703
-0.19138221442699432 0.18119533753702 22.1102
-0.19139738380908966 0.18150788548703622 22.1057
-0.19139738380908966 0.18150788548703622 22.1057
-0.04965734854340553 0.2019343704396388 22.2991
-0.09991157054901123 0.20940301949442996 22.0881
-0.09986810386180878 0.19444799152704706 22.2988
-0.09986810386180878 0.194447991

-0.23247312009334564 0.1501352486900649 22.3022
-0.23247312009334564 0.1501352486900649 22.3022
-0.2170257717370987 0.15800631577791668 22.2892
-0.26404574513435364 0.13622776326044395 22.2774
-0.2638537585735321 0.15272529898262033 22.0462
-0.2638537585735321 0.15272529898262033 22.0462


127.0.0.1 - - [20/Jan/2021 13:57:04] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 38.657591
